# Case One: Project Notebook
By August and William

In [1]:
### Imports
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# 1. Data Preprocessing
## Load data and remove nan's

In [2]:
df_full = pd.read_excel('dataset_full.xls')
df_full.FlightNumber = df_full.FlightNumber.astype(object)
df = df_full.dropna()
df = df.loc[df['FlightType'].isin(['C', 'J'])]

## Augment **ScheduleTime** column

In [3]:
df['Year'] = df.ScheduleTime.dt.year.astype(object);
df['Month'] = df['ScheduleTime'].dt.month.astype(object);
df['WeekNumber'] = df['ScheduleTime'].dt.isocalendar().week % 52;
df['Weekday'] = df['ScheduleTime'].dt.dayofweek;
df.WeekNumber = df.WeekNumber.astype(object);
df.Weekday = df.Weekday.astype(object);
df['HourOfDay'] = df['ScheduleTime'].dt.hour.astype(object);
df['MinuteOfHour'] = df['ScheduleTime'].dt.minute.astype(object);

## One-Hot-Encode nominal variables

In [4]:
### Define feature columns
# feature_cols = ['ScheduleTime', 'Airline', 'FlightNumber', 'Destination', 'AircraftType', 'FlightType', 'Sector', 'SeatCapacity', 'Year', 'Month', 'WeekNumber', 'Weekday', 'HourOfDay', 'MinuteOfHour']
feature_cols = ['Airline', 'FlightNumber', 'Destination', 'AircraftType', 'FlightType', 'Sector', 'SeatCapacity', 'Year', 'Month', 'WeekNumber', 'Weekday', 'HourOfDay', 'MinuteOfHour']
under_15_cols = ['FlightType', 'Sector', 'Year', 'Month', 'Weekday', 'MinuteOfHour']
over_15_cols = ['Airline', 'FlightNumber', 'Destination', 'AircraftType', 'WeekNumber', 'HourOfDay']
nominal_cols = ['Airline', 'FlightNumber', 'Destination', 'AircraftType', 'FlightType', 'Sector', 'Year', 'Month', 'WeekNumber', 'Weekday', 'HourOfDay', 'MinuteOfHour']
ordinal_cols = ['SeatCapacity']

### Split target from feature data
X_full = df[feature_cols]
y = df['LoadFactor']

### Encode features with one-hot-encoding
# X = pd.get_dummies(data=X, columns=under_15_cols)
X_full = pd.get_dummies(data=X_full, columns=nominal_cols);

### Print dataframe
X_full

,SeatCapacity,Airline_5M,Airline_AY,Airline_BJ,Airline_BT,Airline_BZ,Airline_CD,Airline_CL,Airline_CN,Airline_DO,...,MinuteOfHour_15,MinuteOfHour_20,MinuteOfHour_25,MinuteOfHour_30,MinuteOfHour_35,MinuteOfHour_40,MinuteOfHour_45,MinuteOfHour_50,MinuteOfHour_54,MinuteOfHour_55
0,142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,74,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,142,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,186,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39445,156,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
39446,98,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39447,186,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
### Print shape of target
y

0        0.408451
1        0.189189
2        0.570423
3        0.333333
4        0.204301
           ...   
39444    0.847222
39445    0.871795
39446    0.857143
39447    0.682796
39448    0.820513
Name: LoadFactor, Length: 39446, dtype: float64

We now have the following objects:  
- **X_full** containing all training data, 1312 features (without ScheduleTime).  
- **y** with target data.  
- **X_SeatCapacity** with original seat numbers.  

# 2. Feature Selection
## Berid training data of insignificant features

In [6]:
### We use the variance threshold method for removing features
from sklearn.feature_selection import VarianceThreshold, SelectFdr
FEATURE_SELECTION_VARIANCE_THRESHOLD = 0.005
FEATURE_SELECTION_BH_ALPHA = 0.00005

selector = VarianceThreshold(FEATURE_SELECTION_VARIANCE_THRESHOLD)
X = pd.DataFrame(selector.fit_transform(X_full, y), columns=selector.get_feature_names_out())
X

,SeatCapacity,Airline_AY,Airline_CL,Airline_CN,Airline_DO,Airline_EM,Airline_GQ,Airline_IA,Airline_IK,Airline_IR,...,MinuteOfHour_10,MinuteOfHour_15,MinuteOfHour_20,MinuteOfHour_25,MinuteOfHour_30,MinuteOfHour_35,MinuteOfHour_40,MinuteOfHour_45,MinuteOfHour_50,MinuteOfHour_55
0,142,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,74,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,142,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,186,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39441,144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
39442,156,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39443,98,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39444,186,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [7]:
y

0        0.408451
1        0.189189
2        0.570423
3        0.333333
4        0.204301
           ...   
39444    0.847222
39445    0.871795
39446    0.857143
39447    0.682796
39448    0.820513
Name: LoadFactor, Length: 39446, dtype: float64

### Now, the training data has been reduced from1312 features to 254 features.

## Make copy of **SeatCapacity** for computing forecast accuracy

In [8]:
# SeatCapacity = df.SeatCapacity
X['SeatCapacityOriginal'] = X.SeatCapacity
X

,SeatCapacity,Airline_AY,Airline_CL,Airline_CN,Airline_DO,Airline_EM,Airline_GQ,Airline_IA,Airline_IK,Airline_IR,...,MinuteOfHour_15,MinuteOfHour_20,MinuteOfHour_25,MinuteOfHour_30,MinuteOfHour_35,MinuteOfHour_40,MinuteOfHour_45,MinuteOfHour_50,MinuteOfHour_55,SeatCapacityOriginal
0,142,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,142
1,74,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,74
2,142,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,142
3,72,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,72
4,186,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39441,144,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,144
39442,156,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,156
39443,98,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,98
39444,186,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,186


# 3. Data splitting
## Split data into modeling data (will be training and validation) and test data

In [9]:
from sklearn.model_selection import train_test_split

### Make train/val set *0.8 and test *0.2
def split_model_test(X, y, seed=0, shuffle=False):
    X_model, X_test, y_model, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, shuffle=shuffle)
    return X_model, X_test, y_model, y_test

def split_train_val(X_m, y_m, seed=0, shuffle=False):
    X_train, X_val, y_train, y_val = train_test_split(X_m, y_m, test_size=0.25, random_state=seed, shuffle=shuffle)
    return X_train, X_val, y_train, y_val

def seperate_SCO(X_train_model, X_val_test):
    X_train_model_SCO, X_val_test_SCO = X_train_model.SeatCapacityOriginal, X_val_test.SeatCapacityOriginal

    X_train_model = X_train_model.loc[:, ~X_train_model.columns.isin(['SeatCapacityOriginal'])]
    X_val_test = X_val_test.loc[:, ~X_val_test.columns.isin(['SeatCapacityOriginal'])]

    return X_train_model, X_val_test, X_train_model_SCO, X_val_test_SCO


# 4. Define validation setup for different models
## Define forecast accuracy function

In [10]:
def mean_forecast_accuracy(loadfactor_forecasted, loadfactor_true, seatcapacity):

    passengers_true = loadfactor_true * seatcapacity
    passengers_forecasted = loadfactor_forecasted * seatcapacity
    
    abs_deviation_per_flight = np.abs((passengers_true-passengers_forecasted) / passengers_true)
    abs_deviation_per_flight[abs_deviation_per_flight >= 10000] = 100

    mean_forecast_acc = np.mean(1 - abs_deviation_per_flight*1)*100
    # print(f'Mean forecast accuracy = {mean_forecast_acc}')
    return mean_forecast_acc

## Define normalizer for training on **SeatCapacity**

In [11]:
def normalize_seatcapacity_fit(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train.SeatCapacity.values.reshape(-1, 1))
    X_train.SeatCapacity = scaler.transform(X_train.SeatCapacity.values.reshape(-1, 1))
    return X_train, scaler

def normalize_seatcapacity(X_val, scaler):
    X_val.SeatCapacity = scaler.transform(X_val.SeatCapacity.values.reshape(-1, 1))
    return X_val

## Functions for fitting+validating models, as well as testing models

In [24]:
### Make function for fitting and validating model
def fit_evaluate_model(X_tr_m, X_v_te, y_tr_m, y_v_te, model):
    
    ## Remove original seatcapacity
    X_tr_m, X_v_te, X_tr_m_SCO, X_v_te_SCO = seperate_SCO(X_train_model=X_tr_m, X_val_test=X_v_te)

    ## Normalize seatcapacity
    X_tr_m, fitted_scaler = normalize_seatcapacity_fit(X_train=X_tr_m)
    ## Fit model to the training data
    model.fit(X=X_tr_m, y=y_tr_m)

    ## Normalize validation data SeatCapacity for predictions
    X_v_te = normalize_seatcapacity(X_val=X_v_te, scaler=fitted_scaler)
    ## Make predictions
    pred = model.predict(X_v_te)

    ## Compute forecasting accuracy
    acc = mean_forecast_accuracy(loadfactor_forecasted=pred, loadfactor_true=y_v_te.to_numpy(), seatcapacity=X_v_te_SCO.to_numpy())

    return acc, model

# Compute EPE ground function

In [13]:
# shuffle = True
# M = 100
# forecast_acc = []

# for m in range(M):

#     ## Split data
#     seed = np.random.randint(10000)
#     X_model, X_test, y_model, y_test = split_model_test(X=X, y=y, seed=seed, shuffle=shuffle)
#     X_train, X_val, y_train, y_val = split_train_val(X_m=X_model, y_m=y_model, seed=seed, shuffle=shuffle)

#     ## Train model on training data with different model parameters
#     #TODO MODEL TRAINING FUNCTION
#     val_accs, best_model = 

#     ## Evaluate best model on test data
#     test_acc, _ = fit_evaluate_model(X_tr_m=X_train, X_v_te=X_test, y_tr_m=y_train, y_v_te=y_test, model=best_model)
#     forecast_acc.append(test_acc)


# Elastic Net

In [14]:
def find_best_elastic_model(X_train, X_val, y_train, y_val):
    
    val_accs = {}
    alphas = np.linspace(0,20,20)
    for a in alphas[1:]:
        model = ElasticNet(alpha=a, l1_ratio=0.5)
        val_acc, model = fit_evaluate_model(X_tr_m=X_train, X_v_te=X_val, y_tr_m=y_train, y_v_te=y_val, model=model)
        val_accs[a] = val_acc


    return val_accs

In [15]:
seed = 1
shuffle = True
X_model, X_test, y_model, y_test = split_model_test(X=X, y=y, seed=seed, shuffle=shuffle)
X_train, X_val, y_train, y_val = split_train_val(X_m=X_model, y_m=y_model, seed=seed, shuffle=shuffle)
accs = find_best_elastic_model(X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val)

accs

{1.0526315789473684: -42.31303067921718,
 2.1052631578947367: -41.99731061421685,
 3.1578947368421053: -41.960908325347205,
 4.2105263157894735: -41.960908325347205,
 5.263157894736842: -41.960908325347205,
 6.315789473684211: -41.960908325347205,
 7.368421052631579: -41.960908325347205,
 8.421052631578947: -41.960908325347205,
 9.473684210526315: -41.960908325347205,
 10.526315789473683: -41.960908325347205,
 11.578947368421051: -41.960908325347205,
 12.631578947368421: -41.960908325347205,
 13.68421052631579: -41.960908325347205,
 14.736842105263158: -41.960908325347205,
 15.789473684210526: -41.960908325347205,
 16.842105263157894: -41.960908325347205,
 17.894736842105264: -41.960908325347205,
 18.94736842105263: -41.960908325347205,
 20.0: -41.960908325347205}

In [16]:
shuffle = True
M = 100
forecast_acc = []

for m in range(M):

    ## Split data
    seed = np.random.randint(10000)
    X_model, X_test, y_model, y_test = split_model_test(X=X, y=y, seed=seed, shuffle=shuffle)
    X_train, X_val, y_train, y_val = split_train_val(X_m=X_model, y_m=y_model, seed=seed, shuffle=shuffle)

    ## Train model on training data with different model parameters
    #TODO MODEL TRAINING FUNCTION
    # val_accs, best_model = 
    model = ElasticNet(alpha=1, l1_ratio=0.5)

    ## Evaluate best model on test data
    test_acc, _ = fit_evaluate_model(X_tr_m=X_train, X_v_te=X_test, y_tr_m=y_train, y_v_te=y_test, model=model)
    forecast_acc.append(test_acc)


print(f'Mean of test accuracies: {np.mean(forecast_acc)}\nStd. of test accuracies: {np.var(forecast_acc)}')

Mean of test accuracies: -36.93078545910262
Std. of test accuracies: 38.302976932198796


# Random Forest

# Gradient Boosted Trees

In [17]:
def find_best_xgboost_model(X_train, X_val, y_train, y_val):
    
    val_accs = {}
    alphas = np.linspace(0,20,20)
    for a in alphas[1:]:
        model = XGBRegressor(max_depth=int(a))
        val_acc, model = fit_evaluate_model(X_tr_m=X_train, X_v_te=X_val, y_tr_m=y_train, y_v_te=y_val, model=model)
        val_accs[a] = val_acc


    return val_accs, max(val_accs, key=val_accs.get)

seed = 1
shuffle = False
X_model, X_test, y_model, y_test = split_model_test(X=X, y=y, seed=seed, shuffle=shuffle)
X_train, X_val, y_train, y_val = split_train_val(X_m=X_model, y_m=y_model, seed=seed, shuffle=shuffle)
accs, max_acc = find_best_xgboost_model(X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val)

accs

{1.0526315789473684: 19.58984204460955,
 2.1052631578947367: 21.058953759016994,
 3.1578947368421053: 22.283512332578965,
 4.2105263157894735: 23.958080410210307,
 5.263157894736842: 25.971673017382198,
 6.315789473684211: 24.767662321526526,
 7.368421052631579: 25.529127127637107,
 8.421052631578947: 28.142643562870017,
 9.473684210526315: 25.737688864517487,
 10.526315789473683: 25.056291342813825,
 11.578947368421051: 26.823367295982216,
 12.631578947368421: 27.352924724919692,
 13.68421052631579: 25.958576058437455,
 14.736842105263158: 27.496819283086943,
 15.789473684210526: 28.383813965072633,
 16.842105263157894: 26.273853005103508,
 17.894736842105264: 27.390566100573782,
 18.94736842105263: 26.66629320010499,
 20.0: 27.852899595919347}

In [18]:
shuffle = True
M = 10
forecast_acc = []

for m in range(M):

    ## Split data
    seed = np.random.randint(10000)
    X_model, X_test, y_model, y_test = split_model_test(X=X, y=y, seed=seed, shuffle=shuffle)
    X_train, X_val, y_train, y_val = split_train_val(X_m=X_model, y_m=y_model, seed=seed, shuffle=shuffle)
    # print(f'Shape X_model = {X_train.shape}')

    ## Train model on training data with different model parameters
    #TODO MODEL TRAINING FUNCTION
    # val_accs, best_model = 
    model = XGBRegressor()

    ## Evaluate best model on test data
    test_acc, _ = fit_evaluate_model(X_tr_m=X_train, X_v_te=X_test, y_tr_m=y_train, y_v_te=y_test, model=model)
    forecast_acc.append(test_acc)


print(f'Mean of test accuracies: {np.mean(forecast_acc)}\nStd. of test accuracies: {np.var(forecast_acc)}')

Mean of test accuracies: 33.98719329671349
Std. of test accuracies: 14.867881822567838


# Make WandB Sweeps to preliminarily find best ranges for parameters

In [26]:
import wandb
wandb_config = {
      'max_depth': 20, 
      'tree_method': 'gpu_hist'
  }

## Initialize WandB for logging config and metrics
wandb.init(project='02582_case1_1', entity='tgml', config=wandb_config)

## Split data
seed = np.random.randint(10000)
X_model, X_test, y_model, y_test = split_model_test(X=X, y=y, seed=seed, shuffle=shuffle)
X_train, X_val, y_train, y_val = split_train_val(X_m=X_model, y_m=y_model, seed=seed, shuffle=shuffle)

## Train model on training data with different model parameters
model = XGBRegressor(max_depth=wandb_config['max_depth'], tree_method=wandb_config['tree_method'])

model_on_test_acc, _ = fit_evaluate_model(X_tr_m=X_model, X_v_te=X_test, y_tr_m=y_model, y_v_te=y_test, model=model)

## Evaluate best model on test data
train_on_test_acc, _ = fit_evaluate_model(X_tr_m=X_train, X_v_te=X_test, y_tr_m=y_train, y_v_te=y_test, model=model)

wandb.log({'model_on_test_acc':model_on_test_acc, 'train_on_test_acc': train_on_test_acc})


model_on_test_acc,▁
train_on_test_acc,▁
model_on_test_acc,39.24715
train_on_test_acc,37.26646


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
